# RSL (Relative Stärke Levy) Aktien-Screening-Strategie

## Überblick
Dieses Notebook implementiert die **Relative Stärke Levy (RSL)** Strategie zur Bewertung und Rangfolge von S&P 500 Aktien.

### Was ist RSL?
Der Relative Stärke Levy Indikator wurde 1967 von Robert Levy entwickelt. Er misst das Momentum einer Aktie, indem er den aktuellen Kurs mit dem historischen Durchschnitt vergleicht.

**Formel:**
```
RSL = Aktueller Kurs / SMA(Kurs, N Perioden)
```

Wobei:
- **Aktueller Kurs** = Letzter Schlusskurs
- **SMA** = Einfacher gleitender Durchschnitt über N Perioden
- **N** = Typischerweise 27 Wochen (~130 Handelstage) oder 52 Wochen (~260 Handelstage)

### Interpretation:
- **RSL > 1,0**: Aktie handelt über ihrem Durchschnitt (bullisches Momentum)
- **RSL < 1,0**: Aktie handelt unter ihrem Durchschnitt (bärisches Momentum)
- **Höherer RSL**: Stärkere relative Stärke

### Strategie:
Kaufe Aktien in den oberen 25% nach RSL-Ranking (Aktien mit stärkstem Momentum).

---

## 1. Installation
Erforderliche Bibliotheken installieren (diese Zelle zuerst in Google Colab ausführen)

In [ ]:
# Erforderliche Pakete installieren
!pip install yfinance pandas openpyxl beautifulsoup4 lxml tqdm xlsxwriter --quiet
print("Alle Pakete erfolgreich installiert!")

## 2. Bibliotheken importieren

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import warnings
import os

# Warnungen unterdrücken für sauberere Ausgabe
warnings.filterwarnings('ignore')

# Anzeigeeinstellungen
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', None)

print("Bibliotheken erfolgreich importiert!")
print(f"Ausführungsdatum: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")

## 3. Konfigurationsparameter
Passen Sie diese Parameter an, um die Analyse anzupassen

In [ ]:
# =============================================================================
# KONFIGURATION
# =============================================================================

# RSL-Berechnungszeitraum (in Handelstagen)
# 27 Wochen = ~130 Handelstage (Levys Original)
# 52 Wochen = ~260 Handelstage (1 Jahr)
RSL_PERIODE = 130  # 27 Wochen / ~6 Monate

# Datenabruf-Zeitraum (zusätzliche Tage für SMA-Berechnung benötigt)
RUECKBLICK_TAGE = 365  # 1 Jahr Daten für ausreichende Historie

# Rate-Limiting (Sekunden zwischen API-Aufrufen)
API_VERZOEGERUNG = 0.1  # 100ms Verzögerung zwischen Anfragen

# Schwellenwert für Top-Performer
TOP_PROZENT = 0.25  # Obere 25%

# Name der Ausgabedatei
AUSGABE_DATEI = f"RSL_Rangliste_{datetime.now().strftime('%Y%m%d')}.xlsx"

print("Konfiguration geladen:")
print(f"  - RSL-Periode: {RSL_PERIODE} Handelstage (~{RSL_PERIODE//5} Wochen)")
print(f"  - Rückblick-Zeitraum: {RUECKBLICK_TAGE} Tage")
print(f"  - Top-Performer: Obere {int(TOP_PROZENT*100)}%")
print(f"  - Ausgabedatei: {AUSGABE_DATEI}")

## 4. S&P 500 Ticker von Wikipedia abrufen
Aktuelle Liste der S&P 500 Unternehmen laden

In [ ]:
def hole_sp500_ticker():
    """
    S&P 500 Unternehmensticker und -namen von Wikipedia abrufen.
    
    Rückgabe:
        DataFrame mit Spalten: Symbol, Unternehmen, Sektor, Branche
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    print("Lade S&P 500 Ticker von Wikipedia...")
    
    try:
        # Requests mit Headers verwenden um Blockierung zu vermeiden
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # HTML parsen
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find('table', {'id': 'constituents'})
        
        if table is None:
            # Fallback: erste Tabelle mit erwarteter Struktur finden
            tables = soup.find_all('table', {'class': 'wikitable'})
            for t in tables:
                if t.find('th', string=lambda x: x and 'Symbol' in x):
                    table = t
                    break
        
        # Tabelle in DataFrame einlesen
        df = pd.read_html(str(table))[0]
        
        # Spaltennamen standardisieren
        df.columns = df.columns.str.strip()
        
        # Relevante Spalten extrahieren
        result = pd.DataFrame({
            'Symbol': df['Symbol'].str.strip().str.replace('.', '-', regex=False),  # Yahoo verwendet - statt .
            'Unternehmen': df['Security'].str.strip(),
            'Sektor': df['GICS Sector'].str.strip() if 'GICS Sector' in df.columns else 'K.A.',
            'Branche': df['GICS Sub-Industry'].str.strip() if 'GICS Sub-Industry' in df.columns else 'K.A.'
        })
        
        print(f"Erfolgreich {len(result)} S&P 500 Ticker geladen!")
        return result
        
    except Exception as e:
        print(f"Fehler beim Laden der S&P 500 Liste: {e}")
        print("Verwende Fallback-Tickerliste...")
        
        # Fallback: minimale Liste für Tests
        fallback_ticker = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 
                          'BRK-B', 'UNH', 'JNJ', 'V', 'XOM', 'JPM', 'PG', 'MA']
        return pd.DataFrame({
            'Symbol': fallback_ticker,
            'Unternehmen': ['Apple Inc.', 'Microsoft Corp.', 'Alphabet Inc.', 'Amazon.com Inc.',
                       'NVIDIA Corp.', 'Meta Platforms', 'Tesla Inc.', 'Berkshire Hathaway',
                       'UnitedHealth Group', 'Johnson & Johnson', 'Visa Inc.', 'Exxon Mobil',
                       'JPMorgan Chase', 'Procter & Gamble', 'Mastercard Inc.'],
            'Sektor': ['Technologie'] * 15,
            'Branche': ['Diverse'] * 15
        })

# Ticker abrufen
sp500_df = hole_sp500_ticker()
print(f"\nErste 10 Ticker:")
print(sp500_df.head(10).to_string(index=False))

## 5. RSL-Berechnungsfunktionen definieren

In [ ]:
def berechne_rsl(kurse, periode=130):
    """
    Berechnet die Relative Stärke Levy (RSL) für eine Kursreihe.
    
    RSL = Aktueller Kurs / SMA(Kurs, Periode)
    
    Parameter:
        kurse: Serie von Schlusskursen
        periode: Anzahl der Perioden für SMA-Berechnung
    
    Rückgabe:
        float: RSL-Wert, oder None bei Berechnungsfehler
    """
    if kurse is None or len(kurse) < periode:
        return None
    
    try:
        # Aktuellen Kurs abrufen (neuester)
        aktueller_kurs = kurse.iloc[-1]
        
        # Einfachen gleitenden Durchschnitt berechnen
        sma = kurse.iloc[-periode:].mean()
        
        # Division durch Null vermeiden
        if sma == 0 or pd.isna(sma):
            return None
        
        # RSL berechnen
        rsl = aktueller_kurs / sma
        
        return round(rsl, 4)
        
    except Exception:
        return None


def hole_aktiendaten(ticker, start_datum, end_datum):
    """
    Historische Aktiendaten von Yahoo Finance abrufen.
    
    Parameter:
        ticker: Aktien-Tickersymbol
        start_datum: Startdatum für Datenabruf
        end_datum: Enddatum für Datenabruf
    
    Rückgabe:
        dict mit Kursdaten und Metadaten, oder None bei Fehler
    """
    try:
        # Daten abrufen
        aktie = yf.Ticker(ticker)
        historie = aktie.history(start=start_datum, end=end_datum, auto_adjust=True)
        
        if historie.empty or len(historie) < RSL_PERIODE:
            return None
        
        # Schlusskurse abrufen
        schlusskurse = historie['Close']
        
        # RSL berechnen
        rsl = berechne_rsl(schlusskurse, RSL_PERIODE)
        
        if rsl is None:
            return None
        
        # Zusätzliche Kennzahlen berechnen
        aktueller_kurs = schlusskurse.iloc[-1]
        kurs_52w_hoch = schlusskurse.max()
        kurs_52w_tief = schlusskurse.min()
        
        # Prozent vom 52-Wochen-Hoch berechnen
        proz_vom_hoch = ((aktueller_kurs - kurs_52w_hoch) / kurs_52w_hoch) * 100
        
        # Kursänderungen in Prozent berechnen
        if len(schlusskurse) >= 20:
            kurs_vor_1m = schlusskurse.iloc[-20] if len(schlusskurse) >= 20 else schlusskurse.iloc[0]
            aenderung_1m = ((aktueller_kurs - kurs_vor_1m) / kurs_vor_1m) * 100
        else:
            aenderung_1m = 0
            
        if len(schlusskurse) >= 60:
            kurs_vor_3m = schlusskurse.iloc[-60]
            aenderung_3m = ((aktueller_kurs - kurs_vor_3m) / kurs_vor_3m) * 100
        else:
            aenderung_3m = 0
            
        if len(schlusskurse) >= 130:
            kurs_vor_6m = schlusskurse.iloc[-130]
            aenderung_6m = ((aktueller_kurs - kurs_vor_6m) / kurs_vor_6m) * 100
        else:
            aenderung_6m = 0
        
        return {
            'RSL': rsl,
            'Aktueller_Kurs': round(aktueller_kurs, 2),
            '52W_Hoch': round(kurs_52w_hoch, 2),
            '52W_Tief': round(kurs_52w_tief, 2),
            'Proz_vom_Hoch': round(proz_vom_hoch, 2),
            'Aenderung_1M': round(aenderung_1m, 2),
            'Aenderung_3M': round(aenderung_3m, 2),
            'Aenderung_6M': round(aenderung_6m, 2),
            'Datenpunkte': len(schlusskurse)
        }
        
    except Exception as e:
        return None

print("RSL-Berechnungsfunktionen erfolgreich definiert!")

## 6. Daten abrufen und RSL für alle Aktien berechnen
Dies kann aufgrund der API-Ratenbegrenzung mehrere Minuten dauern

In [ ]:
def verarbeite_alle_aktien(ticker_df, start_datum, end_datum):
    """
    Alle Aktien verarbeiten: Daten abrufen und RSL berechnen.
    
    Parameter:
        ticker_df: DataFrame mit Symbol, Unternehmen, Sektor, Branche Spalten
        start_datum: Startdatum für Datenabruf
        end_datum: Enddatum für Datenabruf
    
    Rückgabe:
        DataFrame mit RSL-Rangliste
    """
    ergebnisse = []
    fehlgeschlagen = []
    
    print(f"\nVerarbeite {len(ticker_df)} Aktien...")
    print(f"Zeitraum: {start_datum.strftime('%d.%m.%Y')} bis {end_datum.strftime('%d.%m.%Y')}")
    print("-" * 60)
    
    for idx, zeile in tqdm(ticker_df.iterrows(), total=len(ticker_df), desc="Lade Daten"):
        ticker = zeile['Symbol']
        unternehmen = zeile['Unternehmen']
        sektor = zeile['Sektor']
        branche = zeile['Branche']
        
        # Abrufen und berechnen
        daten = hole_aktiendaten(ticker, start_datum, end_datum)
        
        if daten:
            ergebnisse.append({
                'Ticker': ticker,
                'Unternehmen': unternehmen,
                'Sektor': sektor,
                'Branche': branche,
                **daten
            })
        else:
            fehlgeschlagen.append(ticker)
        
        # Ratenbegrenzung
        time.sleep(API_VERZOEGERUNG)
    
    print(f"\n" + "=" * 60)
    print(f"Verarbeitung abgeschlossen!")
    print(f"  - Erfolgreich verarbeitet: {len(ergebnisse)} Aktien")
    print(f"  - Fehlgeschlagen/Übersprungen: {len(fehlgeschlagen)} Aktien")
    
    if fehlgeschlagen and len(fehlgeschlagen) <= 20:
        print(f"  - Fehlgeschlagene Ticker: {', '.join(fehlgeschlagen)}")
    elif fehlgeschlagen:
        print(f"  - Erste 20 fehlgeschlagene: {', '.join(fehlgeschlagen[:20])}...")
    
    # DataFrame erstellen und nach RSL sortieren
    ergebnis_df = pd.DataFrame(ergebnisse)
    ergebnis_df = ergebnis_df.sort_values('RSL', ascending=False).reset_index(drop=True)
    
    # Rang-Spalte hinzufügen
    ergebnis_df.insert(0, 'Rang', range(1, len(ergebnis_df) + 1))
    
    return ergebnis_df, fehlgeschlagen


# Datumsbereich berechnen
end_datum = datetime.now()
start_datum = end_datum - timedelta(days=RUECKBLICK_TAGE)

# Alle Aktien verarbeiten
ergebnis_df, fehlgeschlagene_ticker = verarbeite_alle_aktien(sp500_df, start_datum, end_datum)

print(f"\nRSL-Berechnung für {len(ergebnis_df)} Aktien abgeschlossen!")

## 7. Ergebnisse und Zusammenfassungsstatistiken anzeigen

In [ ]:
# Zusammenfassungsstatistiken anzeigen
print("=" * 70)
print("RSL ZUSAMMENFASSUNG")
print("=" * 70)
print(f"\nAnalysierte Aktien gesamt: {len(ergebnis_df)}")
print(f"\nRSL-Statistiken:")
print(f"  - Durchschnitt RSL: {ergebnis_df['RSL'].mean():.4f}")
print(f"  - Median RSL:       {ergebnis_df['RSL'].median():.4f}")
print(f"  - Maximum RSL:      {ergebnis_df['RSL'].max():.4f} ({ergebnis_df.loc[ergebnis_df['RSL'].idxmax(), 'Ticker']})")
print(f"  - Minimum RSL:      {ergebnis_df['RSL'].min():.4f} ({ergebnis_df.loc[ergebnis_df['RSL'].idxmin(), 'Ticker']})")
print(f"  - Standardabw.:     {ergebnis_df['RSL'].std():.4f}")

# Aktien über/unter 1,0 zählen
ueber_eins = (ergebnis_df['RSL'] > 1.0).sum()
unter_eins = (ergebnis_df['RSL'] < 1.0).sum()
print(f"\nMomentum-Verteilung:")
print(f"  - RSL > 1,0 (bullisch): {ueber_eins} Aktien ({ueber_eins/len(ergebnis_df)*100:.1f}%)")
print(f"  - RSL < 1,0 (bärisch):  {unter_eins} Aktien ({unter_eins/len(ergebnis_df)*100:.1f}%)")

# Perzentil-Schwellenwerte
print(f"\nPerzentil-Schwellenwerte:")
for proz in [25, 50, 75, 90, 95]:
    schwelle = ergebnis_df['RSL'].quantile(proz/100)
    print(f"  - {proz}. Perzentil: RSL >= {schwelle:.4f}")

In [ ]:
# Top 25 Aktien anzeigen
print("\n" + "=" * 70)
print("TOP 25 AKTIEN NACH RSL (Stärkstes Momentum)")
print("=" * 70 + "\n")

top_25_anzeige = ergebnis_df.head(25)[['Rang', 'Ticker', 'Unternehmen', 'RSL', 'Aktueller_Kurs', 
                                       'Aenderung_1M', 'Aenderung_3M', 'Aenderung_6M', 'Proz_vom_Hoch']]
print(top_25_anzeige.to_string(index=False))

In [ ]:
# Untere 10 Aktien anzeigen
print("\n" + "=" * 70)
print("UNTERE 10 AKTIEN NACH RSL (Schwächstes Momentum)")
print("=" * 70 + "\n")

untere_10_anzeige = ergebnis_df.tail(10)[['Rang', 'Ticker', 'Unternehmen', 'RSL', 'Aktueller_Kurs', 
                                          'Aenderung_1M', 'Aenderung_3M', 'Aenderung_6M', 'Proz_vom_Hoch']]
print(untere_10_anzeige.to_string(index=False))

In [ ]:
# Sektoranalyse
print("\n" + "=" * 70)
print("SEKTORANALYSE - Durchschnittlicher RSL nach Sektor")
print("=" * 70 + "\n")

sektor_stats = ergebnis_df.groupby('Sektor').agg({
    'RSL': ['mean', 'median', 'count'],
    'Aenderung_6M': 'mean'
}).round(4)

sektor_stats.columns = ['Durchschn_RSL', 'Median_RSL', 'Anzahl_Aktien', 'Durchschn_6M_Aenderung']
sektor_stats = sektor_stats.sort_values('Durchschn_RSL', ascending=False)

print(sektor_stats.to_string())

## 8. Top 25% Portfolio erstellen (Kaufkandidaten)

In [ ]:
# Top 25% Schwellenwert berechnen
top_schwelle = int(len(ergebnis_df) * TOP_PROZENT)
rsl_schwelle = ergebnis_df.iloc[top_schwelle - 1]['RSL'] if top_schwelle > 0 else ergebnis_df['RSL'].max()

# Top-Performer DataFrame erstellen
top_performer = ergebnis_df.head(top_schwelle).copy()
top_performer['Perzentil'] = 'Obere 25%'

print("=" * 70)
print(f"OBERE {int(TOP_PROZENT*100)}% PORTFOLIO - KAUFKANDIDATEN")
print("=" * 70)
print(f"\nAnzahl Aktien: {len(top_performer)}")
print(f"RSL-Schwellenwert: >= {rsl_schwelle:.4f}")
print(f"\nPortfolio-Statistiken:")
print(f"  - Durchschnitt RSL: {top_performer['RSL'].mean():.4f}")
print(f"  - Durchschnitt 1M-Änderung: {top_performer['Aenderung_1M'].mean():.2f}%")
print(f"  - Durchschnitt 3M-Änderung: {top_performer['Aenderung_3M'].mean():.2f}%")
print(f"  - Durchschnitt 6M-Änderung: {top_performer['Aenderung_6M'].mean():.2f}%")

print(f"\nSektorverteilung:")
sektor_verteilung = top_performer['Sektor'].value_counts()
for sektor, anzahl in sektor_verteilung.items():
    proz = anzahl / len(top_performer) * 100
    print(f"  - {sektor}: {anzahl} Aktien ({proz:.1f}%)")

## 9. Excel-Bericht mit mehreren Blättern erstellen

In [ ]:
def erstelle_excel_bericht(ergebnis_df, top_performer, sektor_stats, fehlgeschlagene_ticker, ausgabe_datei):
    """
    Erstellt einen professionellen Excel-Bericht mit mehreren Blättern und Formatierung.
    """
    print(f"\nErstelle Excel-Bericht: {ausgabe_datei}")
    
    # Excel-Writer mit xlsxwriter-Engine für bessere Formatierung erstellen
    with pd.ExcelWriter(ausgabe_datei, engine='xlsxwriter') as writer:
        workbook = writer.book
        
        # Formate definieren
        header_format = workbook.add_format({
            'bold': True,
            'bg_color': '#1F4E79',
            'font_color': 'white',
            'border': 1,
            'align': 'center',
            'valign': 'vcenter'
        })
        
        number_format = workbook.add_format({'num_format': '#.##0,00', 'border': 1})
        percent_format = workbook.add_format({'num_format': '0,00%', 'border': 1})
        rsl_hoch_format = workbook.add_format({'num_format': '0,0000', 'bg_color': '#C6EFCE', 'border': 1})
        rsl_tief_format = workbook.add_format({'num_format': '0,0000', 'bg_color': '#FFC7CE', 'border': 1})
        rsl_normal_format = workbook.add_format({'num_format': '0,0000', 'border': 1})
        text_format = workbook.add_format({'border': 1})
        
        # =====================================================================
        # Blatt 1: Zusammenfassung
        # =====================================================================
        zusammenfassung_daten = {
            'Kennzahl': [
                'Berichtsdatum',
                'RSL-Periode (Tage)',
                'Analysierte Aktien gesamt',
                'Fehlgeschlagene Ticker',
                '',
                'Durchschnitt RSL',
                'Median RSL',
                'Maximum RSL',
                'Minimum RSL',
                'Standardabweichung RSL',
                '',
                'Aktien mit RSL > 1,0',
                'Aktien mit RSL < 1,0',
                '',
                'Obere 25% Schwellenwert (RSL)',
                'Obere 25% Anzahl'
            ],
            'Wert': [
                datetime.now().strftime('%d.%m.%Y %H:%M'),
                RSL_PERIODE,
                len(ergebnis_df),
                len(fehlgeschlagene_ticker),
                '',
                f"{ergebnis_df['RSL'].mean():.4f}",
                f"{ergebnis_df['RSL'].median():.4f}",
                f"{ergebnis_df['RSL'].max():.4f}",
                f"{ergebnis_df['RSL'].min():.4f}",
                f"{ergebnis_df['RSL'].std():.4f}",
                '',
                f"{(ergebnis_df['RSL'] > 1.0).sum()} ({(ergebnis_df['RSL'] > 1.0).sum()/len(ergebnis_df)*100:.1f}%)",
                f"{(ergebnis_df['RSL'] < 1.0).sum()} ({(ergebnis_df['RSL'] < 1.0).sum()/len(ergebnis_df)*100:.1f}%)",
                '',
                f"{top_performer['RSL'].min():.4f}",
                len(top_performer)
            ]
        }
        zusammenfassung_df = pd.DataFrame(zusammenfassung_daten)
        zusammenfassung_df.to_excel(writer, sheet_name='Zusammenfassung', index=False, startrow=1)
        
        ws_zusammenfassung = writer.sheets['Zusammenfassung']
        ws_zusammenfassung.write(0, 0, 'RSL SCREENING ZUSAMMENFASSUNG', workbook.add_format({'bold': True, 'font_size': 14}))
        ws_zusammenfassung.set_column('A:A', 35)
        ws_zusammenfassung.set_column('B:B', 25)
        
        # =====================================================================
        # Blatt 2: Vollständige Rangliste
        # =====================================================================
        ergebnis_df.to_excel(writer, sheet_name='Vollstaendige_Rangliste', index=False, startrow=0)
        
        ws_voll = writer.sheets['Vollstaendige_Rangliste']
        
        # Header-Format anwenden
        for col_num, value in enumerate(ergebnis_df.columns.values):
            ws_voll.write(0, col_num, value, header_format)
        
        # Spaltenbreiten setzen
        ws_voll.set_column('A:A', 6)   # Rang
        ws_voll.set_column('B:B', 8)   # Ticker
        ws_voll.set_column('C:C', 30)  # Unternehmen
        ws_voll.set_column('D:D', 20)  # Sektor
        ws_voll.set_column('E:E', 25)  # Branche
        ws_voll.set_column('F:F', 10)  # RSL
        ws_voll.set_column('G:N', 14)  # Andere Spalten
        
        # Bedingte Formatierung für RSL-Spalte (Spalte F, Index 5)
        rsl_spalte = 5
        ws_voll.conditional_format(1, rsl_spalte, len(ergebnis_df), rsl_spalte, {
            'type': 'cell',
            'criteria': '>=',
            'value': 1.05,
            'format': rsl_hoch_format
        })
        ws_voll.conditional_format(1, rsl_spalte, len(ergebnis_df), rsl_spalte, {
            'type': 'cell',
            'criteria': '<',
            'value': 0.95,
            'format': rsl_tief_format
        })
        
        # Kopfzeile fixieren
        ws_voll.freeze_panes(1, 0)
        
        # =====================================================================
        # Blatt 3: Obere 25% (Kaufkandidaten)
        # =====================================================================
        top_performer.to_excel(writer, sheet_name='Obere_25%_Kaufliste', index=False, startrow=0)
        
        ws_top = writer.sheets['Obere_25%_Kaufliste']
        
        for col_num, value in enumerate(top_performer.columns.values):
            ws_top.write(0, col_num, value, header_format)
        
        ws_top.set_column('A:A', 6)
        ws_top.set_column('B:B', 8)
        ws_top.set_column('C:C', 30)
        ws_top.set_column('D:D', 20)
        ws_top.set_column('E:E', 25)
        ws_top.set_column('F:O', 14)
        ws_top.freeze_panes(1, 0)
        
        # =====================================================================
        # Blatt 4: Sektoranalyse
        # =====================================================================
        sektor_stats.reset_index().to_excel(writer, sheet_name='Sektoranalyse', index=False, startrow=0)
        
        ws_sektor = writer.sheets['Sektoranalyse']
        ws_sektor.set_column('A:A', 25)
        ws_sektor.set_column('B:E', 20)
        
        # =====================================================================
        # Blatt 5: Fehlgeschlagene Ticker (falls vorhanden)
        # =====================================================================
        if fehlgeschlagene_ticker:
            fehlgeschlagen_df = pd.DataFrame({'Fehlgeschlagene_Ticker': fehlgeschlagene_ticker})
            fehlgeschlagen_df.to_excel(writer, sheet_name='Fehlgeschlagene_Ticker', index=False)
    
    print(f"Excel-Bericht erfolgreich erstellt!")
    print(f"Datei gespeichert: {ausgabe_datei}")
    return ausgabe_datei


# Excel-Bericht generieren
excel_datei = erstelle_excel_bericht(ergebnis_df, top_performer, sektor_stats, fehlgeschlagene_ticker, AUSGABE_DATEI)

## 10. Excel-Datei herunterladen (Google Colab)

In [ ]:
# Datei in Google Colab herunterladen
try:
    from google.colab import files
    files.download(excel_datei)
    print(f"\nDownload gestartet für: {excel_datei}")
except ImportError:
    # Läuft nicht in Colab
    print(f"\nLäuft nicht in Google Colab.")
    print(f"Excel-Datei lokal gespeichert: {os.path.abspath(excel_datei)}")
except Exception as e:
    print(f"\nDownload konnte nicht gestartet werden: {e}")
    print(f"Excel-Datei gespeichert unter: {os.path.abspath(excel_datei)}")

## 11. Optional: Visualisierung

In [ ]:
# Einfache textbasierte Visualisierung (funktioniert ohne matplotlib)
print("\n" + "=" * 70)
print("RSL-VERTEILUNG HISTOGRAMM")
print("=" * 70)

# Histogramm-Klassen erstellen
klassen = [0.7, 0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5]
bezeichnungen = ['<0,8', '0,8-0,9', '0,9-0,95', '0,95-1,0', '1,0-1,05', '1,05-1,1', '1,1-1,2', '1,2-1,3', '>1,3']

# Aktien in jeder Klasse zählen
zaehler = pd.cut(ergebnis_df['RSL'], bins=klassen, labels=bezeichnungen[:-1]).value_counts().sort_index()

# Ausreißer zählen
niedrige_ausreisser = (ergebnis_df['RSL'] < 0.7).sum()
hohe_ausreisser = (ergebnis_df['RSL'] > 1.5).sum()

print("\nRSL-Bereich     | Anzahl | Balken")
print("-" * 50)

if niedrige_ausreisser > 0:
    balken = '#' * (niedrige_ausreisser * 2)
    print(f"< 0,7           | {niedrige_ausreisser:5} | {balken}")

for bezeichnung, anzahl in zaehler.items():
    balken = '#' * (anzahl * 2 // 3)  # Skaliert für Anzeige
    print(f"{bezeichnung:15} | {anzahl:5} | {balken}")

if hohe_ausreisser > 0:
    balken = '#' * (hohe_ausreisser * 2)
    print(f"> 1,5           | {hohe_ausreisser:5} | {balken}")

print("\nLegende: # = ~1,5 Aktien")

In [ ]:
# Optional: Matplotlib-Visualisierung (auskommentieren falls matplotlib verfügbar)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # RSL-Verteilung Histogramm
    axes[0].hist(ergebnis_df['RSL'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='RSL = 1,0')
    axes[0].axvline(x=ergebnis_df['RSL'].quantile(0.75), color='green', linestyle='--', 
                    linewidth=2, label=f'Obere 25% Schwelle')
    axes[0].set_xlabel('RSL-Wert', fontsize=12)
    axes[0].set_ylabel('Anzahl Aktien', fontsize=12)
    axes[0].set_title('RSL-Verteilung der S&P 500 Aktien', fontsize=14)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Durchschnittlicher RSL nach Sektor
    sektor_durchschn = ergebnis_df.groupby('Sektor')['RSL'].mean().sort_values(ascending=True)
    farben = ['green' if x > 1.0 else 'red' for x in sektor_durchschn.values]
    axes[1].barh(range(len(sektor_durchschn)), sektor_durchschn.values, color=farben, alpha=0.7)
    axes[1].set_yticks(range(len(sektor_durchschn)))
    axes[1].set_yticklabels(sektor_durchschn.index, fontsize=10)
    axes[1].axvline(x=1.0, color='black', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Durchschnittlicher RSL', fontsize=12)
    axes[1].set_title('Durchschnittlicher RSL nach Sektor', fontsize=14)
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('RSL_Analyse_Diagramme.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Diagramme gespeichert als 'RSL_Analyse_Diagramme.png'")
    
except ImportError:
    print("Matplotlib nicht verfügbar. Grafische Visualisierung übersprungen.")
    print("Für Diagramme ausführen: !pip install matplotlib")

## 12. Abschlusszusammenfassung

In [ ]:
print("\n" + "=" * 70)
print("RSL SCREENING ABGESCHLOSSEN")
print("=" * 70)
print(f"\nBericht erstellt: {AUSGABE_DATEI}")
print(f"Ausführungszeitpunkt: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")
print(f"\nWichtigste Erkenntnisse:")
print(f"  - Analysierte Aktien gesamt: {len(ergebnis_df)}")
print(f"  - Top-Performer: {ergebnis_df.iloc[0]['Ticker']} ({ergebnis_df.iloc[0]['Unternehmen']}) mit RSL = {ergebnis_df.iloc[0]['RSL']:.4f}")
print(f"  - Schwächste Aktie: {ergebnis_df.iloc[-1]['Ticker']} ({ergebnis_df.iloc[-1]['Unternehmen']}) mit RSL = {ergebnis_df.iloc[-1]['RSL']:.4f}")
print(f"  - Aktien mit bullischem Momentum (RSL > 1,0): {(ergebnis_df['RSL'] > 1.0).sum()}")
print(f"  - Obere 25% Kaufkandidaten: {len(top_performer)} Aktien")
print(f"\nNächste Schritte:")
print(f"  1. Prüfen Sie das Blatt 'Obere_25%_Kaufliste' für potenzielle Investments")
print(f"  2. Berücksichtigen Sie die Sektordiversifikation beim Portfolioaufbau")
print(f"  3. Kombinieren Sie RSL mit Fundamentalanalyse für bessere Ergebnisse")
print(f"  4. Führen Sie dieses Screening regelmäßig durch (wöchentlich/monatlich)")
print("\n" + "=" * 70)

---

## Über die RSL-Strategie

### Historischer Hintergrund
Robert Levy führte das Konzept der Relativen Stärke in seiner Arbeit von 1967 "Relative Strength as a Criterion for Investment Selection" ein. Seine Forschung zeigte, dass Aktien mit hoher relativer Stärke dazu neigen, weiterhin überdurchschnittlich abzuschneiden.

### Wie Sie dieses Screening nutzen

1. **Kaufkandidaten**: Konzentrieren Sie sich auf die oberen 25% der Aktien nach RSL
2. **Vermeiden**: Aktien in den unteren 25% (schwächstes Momentum)
3. **Rebalancing**: Überprüfen und rebalancieren Sie monatlich oder vierteljährlich
4. **Diversifikation**: Verteilen Sie Investments über verschiedene Sektoren

### Einschränkungen

- Vergangenes Momentum garantiert keine zukünftige Performance
- Funktioniert am besten in Trendmärkten, kann in Seitwärtsmärkten unterperformen
- Sollte mit anderen Analysen kombiniert werden (Fundamental, Technisch)
- Transaktionskosten können Renditen bei häufigem Rebalancing schmälern

### Haftungsausschluss
Dieses Tool dient ausschließlich Bildungs- und Forschungszwecken. Führen Sie immer Ihre eigene Recherche durch und ziehen Sie die Beratung eines Finanzberaters in Betracht, bevor Sie Investitionsentscheidungen treffen.

---
*Erstellt mit Python | Daten von Yahoo Finance & Wikipedia*